## Create _C.virginia_ gene BED file

### Resulting gene BED file will be used for [_C.virginica_ RNAseq/DML sex/OA project](https://github.com/epigeneticstoocean/2018_L18-adult-methylation) (GitHub repo)

This notebook relies on [GFFutils](https://gffutils.readthedocs.io/en/v0.12.0/index.html) to be installed and available in your `$PATH`.

### List computer specs

In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Thu 09 Dec 2021 01:47:25 PM PST
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.3 LTS
Release:	20.04
Codename:	focal

------------
HOSTNAME: 
computer

------------
Computer Specs:

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   45 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              1
Core(s) per socket:              1
Socket(s):                       2
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           165
Model name:                      Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz
Stepping:                        2
CPU MHz:                         2400.007
BogoMIPS:                        4800.01
Hypervisor vendor:               VMware
Virtualization type:    

No LSB modules are available.


### Set variables
- `%env` indicates a bash variable

- without `%env` is Python variable

In [2]:
# Set directories, input/output files
%env data_dir=/home/sam/data/C_virginica/igv_tracks
%env analysis_dir=/home/sam/analyses/20211209_cvir_gff-to-bed
analysis_dir="/home/sam/analyses/20211209_cvir_gff-to-bed"

# Input GFF (from NCBI)
%env orig_gff=GCF_002022765.2_C_virginica-3.0_genomic.gff
%env orig_gff_gz=GCF_002022765.2_C_virginica-3.0_genomic.gff.gz
%env orig_gff_url=https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/022/765/GCF_002022765.2_C_virginica-3.0

# GTF extractor output
%env gtf_extractor_output=20211209_cvir_GCF_002022765.2_genes.bed

env: data_dir=/home/sam/data/C_virginica/igv_tracks
env: analysis_dir=/home/sam/analyses/20211209_cvir_gff-to-bed
env: orig_gff=GCF_002022765.2_C_virginica-3.0_genomic.gff
env: orig_gff_gz=GCF_002022765.2_C_virginica-3.0_genomic.gff.gz
env: orig_gff_url=https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/022/765/GCF_002022765.2_C_virginica-3.0
env: gtf_extractor_output=20211209_cvir_GCF_002022765.2_genes.bed


### Download GFF

In [3]:
%%bash
cd "${data_dir}"

# Download with wget.
# Use --quiet option to prevent wget output from printing too many lines to notebook
# Use --continue to prevent re-downloading fie if it's already been downloaded.
wget --quiet \
--continue \
${orig_gff_url}/${orig_gff_gz}

# Unzip download GFF
gunzip "${orig_gff}"

ls -ltrh "${orig_gff}"

-rw-rw-r-- 1 sam sam 412M Dec 10  2019 GCF_002022765.2_C_virginica-3.0_genomic.gff


### Examine GFF

In [4]:
%%bash
head -n 20 "${data_dir}"/"${orig_gff}"

##gff-version 3
#!gff-spec-version 1.21
#!processor NCBI annotwriter
#!genome-build C_virginica-3.0
#!genome-build-accession NCBI_Assembly:GCF_002022765.2
#!annotation-source NCBI Crassostrea virginica Annotation Release 100
##sequence-region NC_035780.1 1 65668440
##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=6565
NC_035780.1	RefSeq	region	1	65668440	.	+	.	ID=NC_035780.1:1..65668440;Dbxref=taxon:6565;Name=1;chromosome=1;collection-date=22-Mar-2015;country=USA;gbkey=Src;genome=chromosome;isolate=RU13XGHG1-28;isolation-source=Rutgers Haskin Shellfish Research Laboratory inbred lines (NJ);mol_type=genomic DNA;tissue-type=whole sample
NC_035780.1	Gnomon	gene	13578	14594	.	+	.	ID=gene-LOC111116054;Dbxref=GeneID:111116054;Name=LOC111116054;gbkey=Gene;gene=LOC111116054;gene_biotype=lncRNA
NC_035780.1	Gnomon	lnc_RNA	13578	14594	.	+	.	ID=rna-XR_002636969.1;Parent=gene-LOC111116054;Dbxref=GeneID:111116054,Genbank:XR_002636969.1;Name=XR_002636969.1;gbkey=ncRNA;gene=LOC111

### Use [GFFutils](https://gffutils.readthedocs.io/en/v0.12.0/index.html) to extract gene features

In [5]:
%%bash
# Make analysis directory, if it doesn't exist
mkdir --parents "${analysis_dir}"

# Extract just gene features
# Extract chrom,start,end,gene=,and strand fields
# "gene=" is the NCBI gene name, in this particular instance
# Specify input as GFF
# Use awk to to insert a "score" column before the strand column ($5)
# and fill new "score" column with arbitrary value of "0"
time \
gtf_extract \
--feature gene \
--fields=chrom,start,end,gene,strand \
--gff ${data_dir}/${orig_gff} \
| awk 'BEGIN{FS=OFS="\t"}{$5 = 0 OFS $5}1' \
> ${analysis_dir}/${gtf_extractor_output}


real	1m1.550s
user	1m0.180s
sys	0m0.390s


#### Check results

In [6]:
%%bash
cd "${analysis_dir}"
ls -ltrh ${gtf_extractor_output}

echo ""

head ${gtf_extractor_output}

-rw-rw-r-- 1 sam sam 1.8M Dec  9 13:49 20211209_cvir_GCF_002022765.2_genes.bed

NC_035780.1	13578	14594	LOC111116054	0	+
NC_035780.1	28961	33324	LOC111126949	0	+
NC_035780.1	43111	66897	LOC111110729	0	-
NC_035780.1	85606	95254	LOC111112434	0	-
NC_035780.1	99840	106460	LOC111120752	0	+
NC_035780.1	108305	110077	LOC111128944	0	-
NC_035780.1	151859	157536	LOC111128953	0	+
NC_035780.1	163809	183798	LOC111105691	0	-
NC_035780.1	164820	166793	LOC111105685	0	+
NC_035780.1	169468	170178	LOC111105702	0	-


#### Confirm that [GFFutils](https://gffutils.readthedocs.io/en/v0.12.0/index.html) output seem okay

In [7]:
%%bash
# Count gene features via GFFutils
echo "GFFutils number of extracted genes:"
gtf_extract --feature gene --fields=gene --gff ${data_dir}/${orig_gff} | wc -l

echo ""

# Count gene features via awk
echo "awk number of extracted genes:"
awk '$3 == "gene" { print $0 }' ${data_dir}/${orig_gff} | wc -l


GFFutils number of extracted genes:
38838

awk number of extracted genes:
38838


### Generate checksums

In [8]:
%%bash
cd "${analysis_dir}"

for file in *
do
  md5sum "${file}" | tee --append checksums.md5
done

c61136062edfa2eac851baf3285e3646  20211209_cvir_GCF_002022765.2_genes.bed
